LangGraph Chatbot with Groq (Simple Chatbox)

In [ ]:
# Install required packages (run once)
!pip install langgraph langsmith
!pip install langchain langchain_groq langchain_community

In [ ]:
# Retrieve API keys securely from Colab secrets
from google.colab import userdata
groq_api_key=userdata.get('Grog_Api_Key')
langsmith=userdata.get('langsmith_api_key')
print(groq_api_key)
print(langsmith)

In [ ]:
# Set environment variables for LangSmith (we disable tracing to avoid warnings)
import os
os.environ["langsmith_api_key"] = langsmith
os.environ["LANGCHAIN_TRACING_V2"]="false"
os.environ["LANGCHAIN_PROJECT"]="TestLanggraph"

In [ ]:
# Import the Groq LLM wrapper from LangChain
from langchain_groq import ChatGroq

In [ ]:
# Initialize the LLM using Groq's hosted model (openai/gpt-oss-120b is a large open-source model via Groq)
llm=ChatGroq(groq_api_key=groq_api_key,model_name="openai/gpt-oss-120b")
llm

## Start Building Chatbot Using Langgraph

In [ ]:
## Start Building Chatbot Using LangGraph
from typing import Annotated
from typing_extensions import TypedDict
# Core LangGraph imports
from langgraph.graph import StateGraph,START,END
from langgraph.graph.message import add_messages

In [ ]:
# Define the state structure for the graph
# The state holds a list of messages, and add_messages ensures new messages are appended
class State(TypedDict):
  # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
  messages:Annotated[list,add_messages]

# Create a StateGraph builder instance
graph_builder=StateGraph(State)


In [ ]:
graph_builder

In [ ]:
# Define the chatbot node: takes current state, calls LLM, returns new message
def chatbot(state:State):
  return {"messages":llm.invoke(state['messages'])}

In [ ]:
# Add the chatbot node to the graph
graph_builder.add_node("chatbot",chatbot)

In [ ]:
# Define the flow: START → chatbot → END
graph_builder.add_edge(START,"chatbot")
graph_builder.add_edge("chatbot",END)

In [ ]:
# Compile the graph into an executable app
graph=graph_builder.compile()

In [ ]:
# Optional: Visualize the graph structure (requires mermaid support)
from IPython.display import Image, display
try:
  display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
  pass

In [ ]:
# Simple synchronous chat loop using graph.stream()
while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "q"]:
        print("Good Bye")
        break

    print("Assistant:", end=" ", flush=True)

    for event in graph.stream({"messages": ("user", user_input)}):
        for node_name, node_state in event.items():
            if "messages" in node_state:
                msg = node_state["messages"]
                # Handle both list and single message
                if isinstance(msg, list):
                    msg = msg[-1]
                if hasattr(msg, "content"):
                    print(msg.content, end="", flush=True)

    print("\n" + "-" * 50)

## Astream Events (async) - For real-time token streaming

In [ ]:
import nest_asyncio
nest_asyncio.apply()

import asyncio

async def chat_loop():
    while True:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "q"]:
            print("Good Bye")
            break

        print("Assistant:", end=" ", flush=True)

        async for event in graph.astream_events(
            input={"messages": [("user", user_input)]},
            version="v2"
        ):
            kind = event["event"]
            if kind == "on_chat_model_stream":
                chunk = event["data"]["chunk"]
                if chunk.content:
                    print(chunk.content, end="", flush=True)

        print("\n" + "-" * 50)


asyncio.run(chat_loop())